# 1. tableDraftController



Đây là cách test API bằng cURL cho cả hai controller:

**1. Test tableDraftController:**
```bash
curl -X POST http://localhost:3000/api/draft \
-H "Content-Type: application/json" \
-d '{
    "lesson_id": "lesson123",
    "input": "This is input content",
    "raw": "This is raw content",
    "final": "This is final content"
}'
```

**2. Test tableFeedbackController:**
```bash
curl -X POST http://localhost:3000/api/feedback \
-H "Content-Type: application/json" \
-d '{
    "feedback": "This is feedback content"
}'
```

Hoặc nếu bạn muốn test trực tiếp với Larkbase API:

```bash
curl -X POST http://103.253.20.13:25033/api/larkbase/create-many-records \
-H "Content-Type: application/json" \
-d '{
    "config": {
        "app_id": "cli_a7852e8dc6fc5010",
        "app_secret": "6SIj0RfQ0ZwROvUhkjAwLebhLfJkIwnT",
        "app_base_token": "FjRbbDy10aGpKfso9uxl646Gguc",
        "base_table_id": "tblpnbjrjfMu2wJY"
    },
    "records": [
        {
            "fields": {
                "lesson_id": "lesson123",
                "input": "This is input content",
                "raw": "This is raw content",
                "final": "This is final content"
            }
        }
    ]
}'
```

Nếu bạn đang sử dụng Windows Command Prompt, hãy thay đổi dấu `\` thành `^`:
```bash
curl -X POST http://localhost:3000/api/draft ^
-H "Content-Type: application/json" ^
-d "{\"lesson_id\":\"lesson123\",\"input\":\"This is input content\",\"raw\":\"This is raw content\",\"final\":\"This is final content\"}"
```

# 2. tableLearningMeaningController






Đây là các lệnh cURL để test `tableLearningMeaningTracking.js`:

**1. Test với đầy đủ data:**
```bash
curl -X POST http://localhost:3000/api/submit-learning-meaning \
-H "Content-Type: application/json" \
-d '{
    "meaning_id": "meaning_2200_01112024",
    "lesson_id": "business_meeting_2200_01112024",
    "raw": [
        {
            "question": "What do you like to do at night?",
            "structure": "I like to go ____ at night.",
            "main phrase": "for a walk",
            "optional phrase 1": "to the park",
            "optional phrase 2": "to the movies",
            "question-vi": "Bạn thích làm gì vào ban đêm?",
            "structure-vi": "Tôi thích đi ____ vào ban đêm.",
            "main phrase-vi": "đi dạo",
            "optional phrase 1-vi": "đến công viên",
            "optional phrase 2-vi": "đến rạp chiếu phim"
        }
    ],
    "final": [
        {
            "question": "What do you like to do at night?",
            "structure": "I like to go ____ at night.",
            "main phrase": "for a walk",
            "optional phrase 1": "to the park",
            "optional phrase 2": "to the movies",
            "question-vi": "Bạn thích làm gì vào ban đêm?",
            "structure-vi": "Tôi thích đi ____ vào ban đêm.",
            "main phrase-vi": "đi dạo",
            "optional phrase 1-vi": "đến công viên",
            "optional phrase 2-vi": "đến rạp chiếu phim"
        }
    ]
}'
```

**2. Test với minimal data (chỉ required fields):**
```bash
curl -X POST http://localhost:3000/api/submit-learning-meaning \
-H "Content-Type: application/json" \
-d '{
    "meaning_id": "meaning_2200_01112024",
    "lesson_id": "business_meeting_2200_01112024"
}'
```

**3. Format cho Windows Command Prompt:**
```bash
curl -X POST http://localhost:3000/api/submit-learning-meaning ^
-H "Content-Type: application/json" ^
-d "{\"meaning_id\":\"meaning_2200_01112024\",\"lesson_id\":\"business_meeting_2200_01112024\",\"raw\":[{\"question\":\"What do you like to do at night?\"}],\"final\":[{\"question\":\"What do you like to do at night?\"}]}"
```

**4. Test validation error (thiếu meaning_id):**
```bash
curl -X POST http://localhost:3000/api/submit-learning-meaning \
-H "Content-Type: application/json" \
-d '{
    "lesson_id": "business_meeting_2200_01112024",
    "raw": [],
    "final": []
}'
```

**5. Test validation error (thiếu lesson_id):**
```bash
curl -X POST http://localhost:3000/api/submit-learning-meaning \
-H "Content-Type: application/json" \
-d '{
    "meaning_id": "meaning_2200_01112024",
    "raw": [],
    "final": []
}'
```

Expected responses:

1. Success response:
```json
{
    "message": "Meaning submitted successfully",
    "data": {
        // Response data from Larkbase
    }
}
```

2. Error response (missing required field):
```json
{
    "error": "meaning_id is required"
}
```

3. Error response (server error):
```json
{
    "error": "Failed to submit meaning",
    "details": "error message here"
}
```

Lưu ý:
- Thay `http://localhost:3000` bằng URL thực tế của server
- Raw và final data có thể là array rỗng hoặc string rỗng
- meaning_id và lesson_id là required fields
- Đảm bảo server đang chạy khi test

Dưới đây là cách cập nhật `tableLearningMeaningController.js` để thêm trường `lesson_input` vào đối tượng dữ liệu gửi tới Larkbase:

### Cập Nhật `tableLearningMeaningController.js`

```javascript:backend/src/controllers/tableLearningMeaningController.js
const LarkbaseService = require('../services/larkbaseService');
const { TABLES } = require('../constants/larkbaseTables');

exports.submitMeaning = async (req, res) => {
    try {
        const { meaning_id, lesson_id, lesson_input, raw, final } = req.body;

        // Validate required fields
        if (!meaning_id) {
            return res.status(400).json({
                error: 'meaning_id is required'
            });
        }

        if (!lesson_id || typeof lesson_id !== 'string') {
            return res.status(400).json({
                error: 'lesson_id is required and must be a string'
            });
        }

        if (!lesson_input || typeof lesson_input !== 'string') {
            return res.status(400).json({
                error: 'lesson_input is required and must be a string'
            });
        }

        // Prepare data object matching Larkbase fields structure
        const meaningData = {
            meaning_id: meaning_id,
            lesson_id: lesson_id,
            lesson_input: lesson_input, // Thêm trường lesson_input
            raw: typeof raw === 'string' ? raw : JSON.stringify(raw) || '', // Ensure raw is a string
            final: typeof final === 'string' ? final : JSON.stringify(final) || '' // Ensure final is a string
        };

        console.log('Meaning Data:', meaningData); // Log dữ liệu trước khi gửi

        const response = await LarkbaseService.createRecords(
            meaningData,
            TABLES.MEANING
        );

        res.json({
            message: 'Meaning submitted successfully',
            data: response.data
        });

    } catch (error) {
        console.error('Error submitting meaning:', error);
        res.status(500).json({
            error: 'Failed to submit meaning',
            details: error.message
        });
    }
};
```

### Thay Đổi Chính:
1. **Thêm Trường `lesson_input`**: 
   - Đã thêm trường `lesson_input` vào phần destructuring của `req.body`.
   - Thêm kiểm tra để đảm bảo rằng `lesson_input` là một chuỗi và không bị thiếu.

2. **Cập Nhật Đối Tượng `meaningData`**:
   - Thêm `lesson_input` vào đối tượng `meaningData` trước khi gửi tới Larkbase.

### Kiểm Tra:
- Đảm bảo rằng khi gửi yêu cầu tới API, bạn cũng bao gồm trường `lesson_input` trong body của yêu cầu.
- Ví dụ cURL để kiểm tra:

```bash
curl -X POST http://localhost:3000/api/submit-meaning \
-H "Content-Type: application/json" \
-d '{
    "meaning_id": "meaning_2200_01112024",
    "lesson_id": "business_meeting_2200_01112024",
    "lesson_input": "Input lesson content here",
    "raw": [
        {
            "question": "What do you like to do at night?",
            "structure": "I like to go ____ at night.",
            "main phrase": "for a walk",
            "optional phrase 1": "to the park",
            "optional phrase 2": "to the movies",
            "question-vi": "Bạn thích làm gì vào ban đêm?",
            "structure-vi": "Tôi thích đi ____ vào ban đêm.",
            "main phrase-vi": "đi dạo",
            "optional phrase 1-vi": "đến công viên",
            "optional phrase 2-vi": "đến rạp chiếu phim"
        }
    ],
    "final": [
        {
            "question": "What do you like to do at night?",
            "structure": "I like to go ____ at night.",
            "main phrase": "for a walk",
            "optional phrase 1": "to the park",
            "optional phrase 2": "to the movies",
            "question-vi": "Bạn thích làm gì vào ban đêm?",
            "structure-vi": "Tôi thích đi ____ vào ban đêm.",
            "main phrase-vi": "đi dạo",
            "optional phrase 1-vi": "đến công viên",
            "optional phrase 2-vi": "đến rạp chiếu phim"
        }
    ]
}'
```

### Kết Luận:
Bằng cách thêm trường `lesson_input`, bạn có thể gửi thêm thông tin cần thiết tới Larkbase. Hãy đảm bảo rằng các trường dữ liệu được gửi đúng định dạng và kiểm tra kỹ lưỡng để tránh lỗi.


### Request Body
```json
{
  "lessons": [
    {
      "question": "Which company are you working for?",
      "structure": "I'm the ______ from ABC Company.",
      "phrases": [
        "Sales representative",
        "Sales director", 
        "Sales associate"
      ]
    }
    // Can include multiple lessons
  ]
}
```

### Response
```json
[
  {
    "description": "Hãy nghe mình nói và đọc lại",
    "sentence_hide": "I'm the Sales representative from ABC Company.",
    "sentence_en": "I'm the Sales representative from ABC Company.",
    "sentence_vi": "Tôi là đại diện kinh doanh của công ty ABC."
  },
  {
    "description": "Nhắc lại nhé",
    "sentence_hide": "I'm the _____ ______________ from ABC Company.",
    "sentence_en": "I'm the Sales representative from ABC Company.",
    "sentence_vi": "Tôi là đại diện kinh doanh của công ty ABC."
  },
  // ... total 8 exercise variations per lesson
]
```

### Example cURL
```bash
curl -X POST \
  http://localhost:3000/api/generate-learning-flexible \
  -H 'Content-Type: application/json' \
  -d '{
    "lessons": [
      {
        "question": "Which company are you working for?",
        "structure": "I'\''m the ______ from ABC Company.",
        "phrases": [
          "Sales representative",
          "Sales director",
          "Sales associate"
        ]
      }
    ]
}'
```

### Status Codes
- 200: Success
- 500: Internal Server Error

### Error Response
```json
{
  "error": "Error message here"
}
```

### Notes
- Each lesson generates 8 different exercise variations
- Supports batch processing of multiple lessons
- Uses GPT-4 model with temperature set to 0
- Maximum concurrent requests and batch size are configurable
- 